In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("covid_part2.ipynb")

# Final Project: COVID-19 Dataset
## Exploring COVID-19 Data through Modeling
## Due Date: Thursday, December 13th, 11:59 PM
## Collaboration Policy

Data science is a collaborative activity. While you may talk with other groups about
the project, we ask that you **write your solutions individually**. If you do
discuss the assignments with others outside of your group please **include their names** at the top
of your notebook.


## This Assignment

In this final project, we will investigate COVID-19 data over the past year. This data contains information about COVID-19 case counts, mortalities, vaccination rates, and various other metadata that can assist in modeling various aspects of COVID-19.

Through this final project, you will demonstrate your experience with:
* Data cleaning and EDA using Pandas
* Unsupervised and supervised learning techniques
* Visualization


## Goal

Model and analyze the temporal evolution of COVID-19 mortalities or cases using one unsupervised and one supervised technique of your choice. Interpret your models' results through visualizations, and draw insightful conclusions about the modeling of COVID-19 data.

Recall that we studied linear and logistic regression, decision trees, random forests as part of supervised learning (with labels) and clustering, PCA as part of unsupervised learning (without labels). You are free to use any methods that you find suitable to answer the problem at hand.

In [ ]:
# Run this cell to set up your notebook
import numpy as np
from geopy import *
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import *
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import pearsonr
import re

cases = pd.read_csv('data/time_series_covid19_confirmed_US.csv') # https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv
vaccinations = pd.read_csv('data/people_vaccinated_us_timeline.csv') # https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/people_vaccinated_us_timeline.csv
counties = pd.read_csv('data/co-est2020.csv', encoding='latin-1') # https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/totals/co-est2020.csv
mask_use = pd.read_csv('data/mask-use-by-county.csv') # https://github.com/nytimes/covid-19-data/blob/master/mask-use/mask-use-by-county.csv

<br/><br/><br/>

---

<br/><br/><br/>

## Data Cleaning (Again!)

For this section, please copy over the appropriate answers from your previous notebook submission.

### Part 1: Question 1a

Impute the null values in *all* the datasets with zero values or empty strings where appropriate.

<!--
BEGIN QUESTION
name: q1a
points: 0
-->

In [ ]:
vaccinations = vaccinations.fillna(0)
cases = cases.fillna(0)

In [ ]:
grader.check("q1a")

### Part 1: Question 1d

Generate a valid FIPS code for the `counties` table.

*Hint*: Refer to [this](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) guide on FIPS codes.

<!--
BEGIN QUESTION
name: q1d
points: 0
-->

In [ ]:
state_code = counties['STATE']
sc_str = state_code.map('{:02}'.format).astype(str)
county_code = counties['COUNTY']
cc_str = county_code.map('{:03}'.format).astype(str)
FIPS = sc_str + cc_str
counties['FIPS'] = FIPS

In [ ]:
grader.check("q1d")

### Part 1: Question 1e

Merge the `counties`, `cases`, and `mask_use` tables on an appropriate primary key to generate county-wise data.

<!--
BEGIN QUESTION
name: q1e
points: 0
-->

In [ ]:
counties['PK'] = counties['FIPS'].astype(int)
cases['PK'] = cases['FIPS'].astype(int)
county_data = pd.merge(pd.merge(counties, cases, on = 'PK', how = 'inner'),
mask_use, left_on='PK', right_on='COUNTYFP', how ='inner')
county_data = county_data.drop(columns = ['PK', 'FIPS_y'])
county_data = county_data.rename({'FIPS_x': 'FIPS'}, axis = 'columns')
cases = cases.drop(columns = 'PK')
counties = counties.drop(columns= 'PK')

In [ ]:
grader.check("q1e")

<br/><br/><br/>

---

<br/><br/><br/>

## Question 5: Guided Supervised Modeling

This section will guide you through creating a supervised learning framework that will predict the number of COVID-19 cases per capita given various COVID-19 safety protocols that have been implemented. Then, we will investigate the bias, variance, and observational noise of this framework.

Note that any answer responses without the appropriate work (i.e. code or math) will be subject to additional review and will not receive any credit.

<!-- BEGIN QUESTION -->

### Question 5a

We will use county-wise mask usage data to predict the number of COVID-19 cases on September 12th, 2021. Create a visualization that shows the pairwise correlation between each combination of column in the mask usage data and the number of COVID-19 cases.

*Hint*: You should be plotting 36 correlations.
<!--
BEGIN QUESTION
name: q5a
points: 3
manual: True
-->

In [ ]:
new_table = county_data[['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', '9/12/21']]
dataplot = sns.heatmap(new_table.corr(), cmap="YlGnBu", annot=True)

<!-- END QUESTION -->

### Question 5b

Train a linear regression model to predict the number of COVID-19 cases using county-wise mask usage data for September 12, 2021. Evaluate your model's RMSE on a held-out validation set with 33% of the county-wise data. When possible, make sure to set `random_state = 42` when splitting your data into training and test sets.
<!--
BEGIN QUESTION
name: q5b
points: 5
-->

In [ ]:
X_q5b = county_data[['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']]
y_q5b = county_data['9/12/21']
# Make sure to set random_state = 42 and test_size = 0.33!
X_q5b_train, X_q5b_test, y_q5b_train, y_q5b_test = train_test_split(X_q5b, y_q5b, test_size = 0.33, random_state = 42)
reg = LinearRegression(fit_intercept = True)
reg.fit(X_q5b_train, y_q5b_train)
y_q5b_train_prediction = reg.predict(X_q5b_train)
y_q5b_test_prediction = reg.predict(X_q5b_test)
train_rmse_cases = np.sqrt(np.mean((y_q5b_train - y_q5b_train_prediction)**2))
test_rmse_cases = np.sqrt(np.mean((y_q5b_test - y_q5b_test_prediction)** 2))
train_rmse_cases, test_rmse_cases


In [ ]:
grader.check("q5b")

<!-- BEGIN QUESTION -->

### Question 5c

Explain potential reasons the test set RMSE is much higher as compared to the training set RMSE.
<!--
BEGIN QUESTION
name: q5c
points: 3
manual: True
-->

The test RMSE may be much higher than the training RMSE because the training set is much
larger than the test set. This means that the training set is less succeptible to error.

<!-- END QUESTION -->

### Question 5d

Instead of predicting the number of COVID-19 cases, redo part (b) by predicting the number of cases per capita. Report the model's RMSE on the training and validation set.

Comment on the relationship between the training and test RMSE by predicting the number of cases per capita instead of the total number of cases.
<!--
BEGIN QUESTION
name: q5d
points: 3
-->

In [ ]:
X_q5d = county_data[['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']]
y_q5d = county_data['9/12/21'] / county_data['POPESTIMATE2020']

# Make sure to set random_state = 42 and test_size = 0.33!

X_q5d_train, X_q5d_test, y_q5d_train, y_q5d_test = train_test_split(X_q5d, y_q5d, test_size = 0.33, random_state = 42)
reg1 = LinearRegression(fit_intercept = True)
reg1.fit(X_q5d_train, y_q5d_train)

y_q5d_train_prediction = reg1.predict(X_q5d_train)
y_q5d_test_prediction = reg1.predict(X_q5d_test)

train_rmse_cpc = np.sqrt(np.mean((y_q5d_train - y_q5d_train_prediction)**2))
test_rmse_cpc = np.sqrt(np.mean((y_q5d_test - y_q5d_test_prediction)** 2))

train_rmse_cpc, test_rmse_cpc

# Comment
# The test and train RMSE are much clsoer in value when predicting cases per capita. Here,
# we see that the train and test RMSE are around 0.035 and 0.038 respectively, which are very clsoe
# the train and test RMSE for the total number of cases is 27834 and 62591 which are very far
# in value. This means that predicting the cases per capita is a better choice since it is
# experimentally less susceptible to error.

In [ ]:
grader.check("q5d")

<!-- BEGIN QUESTION -->

### Question 5e

Visualize the model outputs from part (d) by plotting the predictions $\hat{y}$ versus the observations $y$. Comment on what the plot indicates about our linear model as a comment in the code cell.

<!--
BEGIN QUESTION
name: q5e
points: 3
manual: True
-->

In [ ]:
# Scatter plot for yhat versus y.
plt.figure(figsize = (16,10))
plt.scatter(
x = y_q5d_test,
y = y_q5d_test_prediction,
s = 10,
alpha = 0.75)
plt.plot([0.1,0.16], [0.1, 0.16], color='orange')
plt.xlabel('Observed $y$')
plt.ylabel('Predicted $\hat{y}$')
plt.title('Predicted vs Observed Test Data')
plt.plot
# We can see from the plot that there isn't a clear correlation between the observed y and the
# predicted y. It deviates from a diagonal straight line too much (a good model would have
# this shape) so it suggests that this model isn't good enough. However, looking at the data
# points, more points are located on around the imaginary straight line which means that while
# this model isn't extremely accurate, it is also not extremely inaccurate.

<!-- END QUESTION -->

### Question 5f

We will investigate the bias and variance of this improved model on the test set using the bias-variance decomposition to formalize the behaviour of our model. To generate an empirical estimate of the errors and the parameters in the bias-variance decomposition, train 1000 bootstrapped models on the training dataset from part (d).

<!--
BEGIN QUESTION
name: q5f
points: 5
-->

In [ ]:
models = []
X_q5f = county_data[['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']]
y_q5f = county_data['9/12/21'] / county_data['POPESTIMATE2020']
for i in np.arange(1000):
    X_q5f_train, X_q5f_test, y_q5f_train, y_q5f_test = train_test_split(X_q5f, y_q5f, test_size = 0.33)
    X_q5f_train_added_y = X_q5f_train.copy(deep=False)
    X_q5f_train_added_y['y_q5f_train'] = y_q5f_train
    shuffled = X_q5f_train_added_y.sample(n=X_q5f_train_added_y.shape[0], replace=True)

    reg_boot = LinearRegression(fit_intercept = True)
    reg_boot.fit(shuffled[['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']],
        shuffled['y_q5f_train'])
    models.append(reg_boot)

# This is for RMSE, but it is not needed for this question.
y_q5f_train_prediction = reg_boot.predict(X_q5f_train)
train_rmse = np.sqrt(np.mean((y_q5f_train - y_q5f_train_prediction)**2))

In [ ]:
grader.check("q5f")

### Question 5g

To investigate the variance in our test predictions, we sample a particular test point $(x_i, y_i)$ such that $i = 100$. In other words, we will use the 100th point in the test set from part (d), `(X_q5d_test.iloc[100], y_q5d_test.iloc[100])` as the testing point.

Generate predictions and square errors for this test point for all 1000 models, and calculate the *proportion* of the *expected* square error that is captured by the model prediction variance. In other words, we wish to estimate the following quantity:

$$
\frac{\mathrm{Var}(f_\theta(x_i))}{\mathrm{E}_\theta[(y_i - f_\theta(x_i))^2]}
$$

*Hint*: Refer to the bias-variance decomposition from lecture.
<!--
BEGIN QUESTION
name: q5g
points: 5
-->

In [ ]:
actual = y_q5d_test.iloc[100]
predictions = []
ses = []
for model in models:
    pred = model.predict(X_q5d_test)[100]
    se = (pred-actual) ** 2
    predictions.append(pred)
    ses.append(se)
    
prop_var = np.var(predictions) / np.mean(ses)
prop_var

In [ ]:
grader.check("q5g")

<!-- BEGIN QUESTION -->

### Question 5h

Using the bias-variance decomposition, comment on how much the variance of the model contributes to the error on the sample point above. We will extend this scenario to analyze the noise term in the bias-variance decomposition, specifically with regards to this COVID-19 dataset. Consider the following:

i) Assuming no observational noise (i.e. $\epsilon = 0$), what is the *magnitude* of the empirical model bias on this sample point?

ii) Clearly, there is a non-trivial amount of observational noise with COVID-19 case data simply due to how testing works and occurs. Please take a look at [this article](https://fivethirtyeight.com/features/coronavirus-case-counts-are-meaningless/) for more information. Given this infomation, explain the issues with the assumptions and result in 5h(i).

iii) Recall that we typically assume $y = g(x) + \epsilon$, where $\epsilon \sim \mathcal{N}(\mu, \sigma)$. In the theoretical setting for bias-variance, we have assumed $\mu = 0, \sigma > 0$. In this practical setting, analyze and determine how $\epsilon$ could be modeled (as a normal distribution, but you may also consider how it could be modeled as another distribution). Are there any immediate issues with the assumptions we have made in the theoretical setting where $\mu = 0, \sigma > 0$? What conditions on $\mu, \sigma$ could be more appropriate and why?

iv) Does the standard bias-variance decomposition presented in lecture hold given $\epsilon$ being normally distributed according to your answer in part (iii)? If so, please explain why. If not, explain why it does not hold and if possible, how to modify the bias-variance decomposition to make it hold (i.e. perhaps there is an additional noise term $E[\epsilon^3]$). 

*Hint*: Try to express $y = g(x) + \epsilon$ by adding and subtracting a certain quantity.

v) Intuitively, is it possible to minimize the true model bias to 0 given your $\epsilon$ formulation in part (iii)? Why or why not? Justify your answer using part (iv) if possible.

vi) Consider the infinite sample case where an oracle grants you as many samples as requested, and answer the same question in part (v). Is it possible to minimize the true model bias to 0 given your $\epsilon$ formulation in part (iii)? Conclude with an analysis of what our modeling task can approximate using $X\theta \approx y$ in the finite and infinite sample case.

<!--
BEGIN QUESTION
name: q5h
points: 24
manual: True
-->

In [ ]:
(np.mean(ses) - np.var(predictions))**0.5

_(i) The magnitude of the empirical model bias on this sample point can be found using the following formula:
        (np.mean(ses) - np.var(predictions))**0.5
        which in this case equals to around 0.04_
        
_(ii) Given the vast differences in testing procedures and strategies, our calculations and models may not be very accurate. The formula for magnitude of the empirical model bias   used in 5h(i) does not take these differences in testing into account. As a note, the formula for the magnitude is the square root of the difference between the MSE and the variance in predictions. Given the information in the article, we might conclude that either the mean squared error (MSE) or the variance in predictions may not be representative of the actual data, because we did not take into account the differences in the testing procedures, resulting in an inaccurate result._

_(iii) Based on the article, $\mu$ is more likely to be negative ($\mu$ < 0) and it is not equal to 0 as we first assume. This is due to the likely systematic undercounting of COVID testing data, described in the article. This would cause the distribution to be shifted to the left and also cause the mean to be negative._

_(iv) Based on the bias-variance decomposition presented in lecture,_
$E((y - f_{\theta}(x))^{2}) = \sigma^{2} + var(f_{\theta}(x)) + [E(f_{\theta}(x) - g(x))]^{2}$

_and that translates to: model risk = observed variance + model variance + (model bias)$^{2}$_

_This standard bias-variance decomposition does not hold given the new model. Therefore we have to modify some things and it is shown here below._

_We first define y being:_

$y = g(x) + \epsilon$ , where $\epsilon - N(\mu, \sigma^{2})$

_Modifying y with the $\mu$ term that we now know is not zero:_

$y = g(x) + \mu + \epsilon - \mu$

_Then, define:_

$g'(x) = g(x) + \mu$

$\epsilon' = \epsilon - \mu$

_and therefore,_

$y = g'(x) + \epsilon'$

_Applying this to the bias-variance decomposition above, we get:_

$E((y - f_{\theta}(x))^{2}) = \sigma^{2} + var(f_{\theta}(x)) + [E(f_{\theta}(x) - g(x) + \mu)]^{2}$

_So, now we can see here that with the new model with the extra $\mu$ term, the only difference is within the model bias itself. The observed variance and model variance stays the same and the model bias now adopts this extra $+ \mu$ term. This means it is still the same bias but shifted by the value of $\mu$._

_(v) We think that it is not possible to minimize the true model bias to 0 given our $\epsilon$ formulation in part (iii). From part (iv), our g'(x) here is defined as being $g(x) + \mu$ and we have no way to find out the value of $\mu$ (though we know that it is most likely negative, we will never know its exact value). With the value of $\mu$ being unknown, there is no way for us to find out the value of g(x) and therefore we cannot minimize the true model bias to 0._

_(vi) Similar to part (v), as we do not know the value of $\mu$, there is no way for us to find out the value of g(x) and therefore we cannot minimize the true model bias to 0 even with infinite sample cases, let alone finite sample cases._

<!-- END QUESTION -->

### Question 5i

Using the bias-variance decomposition for each test point, calculate the average variance and average test mean-square error across the entire test set from part (d). In other words, estimate the following quantities:

$$
\frac{1}{n} \sum_i \mathrm{Var}(f_\theta(x_i))
$$

and

$$
\frac{1}{n} \sum_i \mathrm{E}_\theta[(y_i - f_\theta(x_i))^2]
$$

<!--
BEGIN QUESTION
name: q5i
points: 5
manual: False
-->

In [ ]:
# creating a dataframe with the 1037 points as the columns (so that we can add the point variance for each model on the rows)
points_as_columns = pd.DataFrame(columns=np.arange(len(predictions)).astype(str))
# adding the rows by the model's evaluation of each of the 1037 points
for model in models:
    pred = model.predict(X_q5d_test)
    model_1000_series = pd.Series(data=pred, index = np.arange(len(pred)).astype(str))
    points_as_columns = points_as_columns.append(model_1000_series, ignore_index = True)
    # calculations
    var = points_as_columns.var()
    actuall = y_q5d_test.to_numpy()
    mses = (points_as_columns.sub(actuall, axis = 'columns') ** 2).mean()
avg_var, avg_mse = np.mean(var), np.mean(mses)
avg_var, avg_mse

In [ ]:
grader.check("q5i")

<!-- BEGIN QUESTION -->

## Question 5j

Propose a solution to reducing the mean square error using the insights gained from the bias-variance decomposition above. What are the values of the quantities that have we estimated and what can be concluded about the remaining quantities? Please show all work that informs your analysis.

Assume that the standard bias-variance decomposition used in lecture can be applied here.
<!--
BEGIN QUESTION
name: q5j
points: 5
manual: True
-->

_Looking at the previous questions, especially 5g, we see that the variance contributes very little to
the model risk (shown by the small value of prop_var in 5g). Therefore, because this model has a
tradeoff between bias and variance and the variance contributes very little, we can conclude that
the bias contributes very largely to the model risk. Thus, we can reduce the mean squared error
by decreasing the model bias. One way to do that is to increase the model complexity as higher
complexity tends to decrease model bias. We can do that by adding more features to be analyzed._

<!-- END QUESTION -->



<br/><br/><br/>

---

<br/><br/><br/>

## Question 6: Open Supervised Modeling

We wish to extend our modeling framework from Question 5 to make it more accurate; in other words, we wish to predict $f(x)$, a supervised learning output, based on past and current quantities. 

This section will serve as a rough guide on creating an autoregressive modeling framework for predicting a COVID-19 quantity of your choice (i.e. deaths, cases, vaccinations).

Note that if you do not wish to pursue time-based modeling of COVID-19, you may skip parts (d), (e), and (f). That being said, you are strongly encouraged to incorporate time-based modeling into your open-ended modeling design since it constitutes a large component of the provided datasets.

We will ***not*** grade these below questions individually (i.e. there are no points explicitly assigned to questions 6(a) to 6(f)); they are simply guiding questions and will be graded as part of the final project report. You should make sure to answer all of the questions (that are applicable to your open-ended modeling) in some form in the report.

### 1.9.1 EDA

In [ ]:
# Read in election data
#votes = pd.read_csv('data/2020_US_County_Level_Presidential_Results.csv', converters = {'county_fips': str}) 
# https://github.com/tonmcg/US_County_Level_Election_Results_08-20/blob/master/2020_US_County_Level_Presidential_Results.csv
#votes.head()


In [ ]:
#import seaborn as sns

#votes = pd.read_csv('data/2020_US_County_Level_Presidential_Results.csv') # Don't need FIPS to be a string anymore

# Generate a merged dataframe (copied from below)
#vaccinations_latest = vaccinations[vaccinations['Date'] == '2021-09-12'][['People_Fully_Vaccinated', 'People_Partially_Vaccinated', 'Province_State']]
#merged = county_data.copy().drop('Province_State', 1)
#merged = pd.merge(merged, vaccinations_latest, how='inner', left_on="STNAME", right_on="Province_State")

# Append our voting data
#merged = merged.astype({'FIPS': 'int64'})
#merged = pd.merge(merged, votes[['county_fips', 'per_point_diff']], how='inner', left_on="FIPS", right_on="county_fips")

# Create a weighted average of vote proportions
#merged['prop_times_pop'] = merged['POPESTIMATE2020'] * merged['per_point_diff']
#vac_votes_df = merged.groupby('STNAME').agg({'prop_times_pop': 'sum', 'POPESTIMATE2020': 'sum'})
#vac_votes_df['weighted_avg'] = vac_votes_df['prop_times_pop'] / vac_votes_df['POPESTIMATE2020']
#vac_votes_df = pd.merge(vac_votes_df, vaccinations_latest, how='inner', left_index=True, right_on='Province_State')
#vac_votes_df['full_vac_prop'] = vac_votes_df['People_Fully_Vaccinated'] / vac_votes_df['POPESTIMATE2020']
#vac_votes_df['part_vac_prop'] = vac_votes_df['People_Partially_Vaccinated'] / vac_votes_df['POPESTIMATE2020']

In [ ]:
#sns.set(rc={'figure.figsize':(11.7,8.27)});
#fig = sns.regplot(x=vac_votes_df['full_vac_prop'], y=vac_votes_df['weighted_avg'])
#fig.set(
#title = 'Proportion Difference in Votes versus Proportion of People Fully Vaccinated in US States',
#xlabel = 'Proportion of People Fully Vaccinated',
#ylabel = 'Proportion Difference in Votes'
#);

In [ ]:
#fig = sns.regplot(x=merged['ALWAYS'], y=merged['per_point_diff'], scatter_kws={'s':3})
#fig.set(
#title = 'Proportion Difference in Votes versus Proportion of People Who Always Wear Masks in US Counties',
#xlabel = 'Proportion of People Who Always Wear Masks',
#ylabel = 'Proportion Difference in Votes'
#);

In [ ]:
#merged['log_cases'] = np.log(merged['9/12/21'])
#cases_df = merged[np.isfinite(merged['log_cases'])]
#fig = sns.regplot(x=cases_df['log_cases'], y=cases_df['per_point_diff'], scatter_kws={'s':3})
#fig.set(
#title = 'Proportion Difference in Votes versus Log Number of Covid Cases on 9/12/21 in US Counties',
#xlabel = 'Log Number of COVID Cases',
#ylabel = 'Proportion Difference in Votes'
#);


In [ ]:
#merged['cases_prop'] = merged['9/12/21'] / merged['POPESTIMATE2020']
#fig = sns.regplot(x=merged['cases_prop'], y=merged['per_point_diff'], scatter_kws={'s':3})
#fig.set(
#title = 'Proportion Difference in Votes versus Proportion of COVID Cases on 9/12/21 in US Counties',
#xlabel = 'Proportion of COVID cases',
#ylabel = 'Proportion Difference in Votes'
#);

In [ ]:
#fig = sns.regplot(x=np.log(merged['POPESTIMATE2020']), y=merged['per_point_diff'], scatter_kws={'s':3})
#fig.set(
#title = 'Proportion Difference in Votes versus Log County Population',
#xlabel = 'Log County Population',
#ylabel = 'Proportion Difference in Votes'
#);

<!-- BEGIN QUESTION -->

### Question 6a

Train a baseline model where $f$ is the model described in Question 0a and $x$ is a quantity of *your* choice. Note that you may used *any* supervised learning approach we have studied; you are not limited to a linear model.

<!--
BEGIN QUESTION
name: q6a
points: 0
manual: True
-->

In [ ]:
# Generate our dataframe with all values
# In order to prevent the feature space from growing too large, we'll use only
# the latest vaccination data rather than vaccination data over time, similar to
# question 3 of part 1 of the project.
#vaccinations_latest = vaccinations[vaccinations['Date'] =='2021-09-12'][['People_Fully_Vaccinated', 'People_Partially_Vaccinated','Province_State']]
#merged = county_data.copy().drop('Province_State', 1)
#merged = pd.merge(merged, vaccinations_latest, how='inner', left_on="STNAME", right_on="Province_State")

# Append our voting data
#merged = merged.astype({'FIPS': 'int64'})
#merged = pd.merge(merged, votes[['county_fips', 'per_point_diff']], how='inner', left_on="FIPS", right_on="county_fips")

# Convert number of vaccinations to proportion as seen in EDA
#grouped = merged.groupby('STNAME', as_index=False).agg({'People_Fully_Vaccinated': 'first', 'People_Partially_Vaccinated':'first', 'POPESTIMATE2020': 'sum'})
#grouped['People_Fully_Vaccinated_Prop'] = grouped['People_Fully_Vaccinated'] / grouped['POPESTIMATE2020']
#grouped['People_Partially_Vaccinated_Prop'] = grouped['People_Partially_Vaccinated'] / grouped['POPESTIMATE2020']
#grouped = grouped.drop(['People_Fully_Vaccinated', 'People_Partially_Vaccinated', 'POPESTIMATE2020'], 1)
#merged = pd.merge(merged, grouped, how='inner', left_on='STNAME', right_on='STNAME')


In [ ]:
# Now we drop all columns that are infeasible or nonsensical for modeling. These consist of mostly identifiers
#data = merged.drop(
#['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'FIPS', 'county_fips', 'COUNTYFP', 'Province_State',
#'UID', 'iso2', 'iso3', 'Admin2', 'Country_Region', 'code3', 'Combined_Key'], 1)

# Rename longitude
#data = data.rename(columns={'Long_': 'Long'})

In [ ]:
# Create our training and testing sets
# Our y value will be the difference in proportion between republican votes and democrat votes (ie republican_votes - democrat_votes)
#X = data.drop('per_point_diff', 1)
#y = data[['per_point_diff']]
#X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

# We drop the proportion for now as it's a linear combination of 2 columns. We'll use it in later models.
#X_train = X_train_full.drop(['People_Fully_Vaccinated_Prop', 'People_Partially_Vaccinated_Prop'], 1)
#X_test = X_test_full.drop(['People_Fully_Vaccinated_Prop', 'People_Partially_Vaccinated_Prop'], 1)

In [ ]:
#from sklearn.metrics import mean_squared_error

# Fit our base model with all feasible features
#base_model = LinearRegression()
#base_model.fit(X_train, y_train)
#y_pred = base_model.predict(X_train)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("Base Model Training RMSE: ", rmse)


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6b

Improve your model from part (a). Specify the supervised model you choose and write $f(x)$ as a function of the chosen features and parameters in your model. Justify why you chose these features and how you expect they will correlate with the output you wish to predict.

<!--
BEGIN QUESTION
name: q6b
points: 0
manual: True
-->

In [ ]:
#from sklearn.linear_model import LassoCV # Use LassoCV to help us choose alpha

# One way to select features is to use LASSO to zero out any redundant features in our feature space
#lasso = LassoCV(cv=5, random_state=0)
#lasso.fit(X_train, y_train)
#lasso_coeffs = lasso.coef_
#lasso_features = X_train.columns[lasso_coeffs > 0]

In [ ]:
#lasso_features

In [ ]:
#X_train_lasso = X_train[lasso_features]
#model_1 = LinearRegression()
#model_1.fit(X_train_lasso, y_train)
#y_pred = model_1.predict(X_train_lasso)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("LASSO Model Training RMSE: ", rmse)

In [ ]:
#from sklearn.linear_model import RidgeCV # Use RidgeCV to help us choose alpha

# We can also try ridge regression as another form of feature penalization
#model_2 = RidgeCV(cv=5, alphas=np.arange(0, 3.01, 0.1))
#model_2.fit(X_train, y_train)
#y_pred = model_2.predict(X_train)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("Ridge Model (No TS) Training RMSE: ", rmse)

In [ ]:
# Time Series data is autocorrelated, so we can try a model that only has covid data from a single date (2021-09-12)
# such that it matches with our vaccination data date and only use a single population estimate.
# Correlation between features generally isn't great for linear regression which is why we're trying this approach.
#wanted_features = [
#'POPESTIMATE2020',
#'Lat',
#'Long',
#'9/12/21',
#'NEVER',
#'RARELY',
#'SOMETIMES',
#'FREQUENTLY',
#'ALWAYS',
#'People_Fully_Vaccinated',
#'People_Partially_Vaccinated'
#]
#X_train_no_ts = X_train[wanted_features]
#model_3 = LinearRegression()
#model_3.fit(X_train_no_ts, y_train)
#y_pred = model_3.predict(X_train_no_ts)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("No Time Series Model Training RMSE: ", rmse)

In [ ]:
# We can once again try LASSO over this smaller feature space
#lasso_no_ts = LassoCV(cv=5, random_state=0)
#lasso_no_ts.fit(X_train_no_ts, y_train)
#lasso_no_ts_coeffs = lasso_no_ts.coef_
#lasso_no_ts_features = X_train_no_ts.columns[lasso_no_ts_coeffs > 0]

In [ ]:
#lasso_no_ts_features

In [ ]:
#X_train_lasso_no_ts = X_train_no_ts[lasso_no_ts_features]
#model_4 = LinearRegression()
#model_4.fit(X_train_lasso_no_ts, y_train)
#y_pred = model_4.predict(X_train_lasso_no_ts)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("LASSO Model (No TS) Training RMSE: ", rmse)

In [ ]:
#from sklearn.linear_model import RidgeCV # Use RidgeCV to help us choose alpha
## We can also try ridge regression as another form of feature penalization
#model_5 = RidgeCV(cv=5, alphas=np.arange(0, 5.01, 0.1))
#model_5.fit(X_train_no_ts, y_train)
#y_pred = model_5.predict(X_train_no_ts)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("Ridge Model (No TS) Training RMSE: ", rmse)

In [ ]:
# Based on EDA from part 1 as well as EDA from this part, we can manually choose variables.
# We try and choose features that are not corrleated with each other at all.
# Additionally, we saw that proportion worked better than raw numbers for vaccination in EDA,
# so we change to using proportions.
#wanted_features_eda = [
#'9/12/21',
#'ALWAYS',
#'People_Fully_Vaccinated_Prop'
#]
#X_train_eda_1 = X_train_full[wanted_features_eda]
#model_6 = LinearRegression()
#model_6.fit(X_train_eda_1, y_train)
#y_pred = model_6.predict(X_train_eda_1)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("EDA Model 1 Training RMSE: ", rmse)

In [ ]:
# Try the same thing but with partially vaccinated proportion rather than fully vaccinated proportion since
# LASSO tends to choose partially vaccinated as an important feature.
#wanted_features_eda = [
#'9/12/21',
#'ALWAYS',
#'People_Partially_Vaccinated_Prop'
#]
#X_train_eda_2 = X_train_full[wanted_features_eda]
#model_7 = LinearRegression()
#model_7.fit(X_train_eda_2, y_train)
#y_pred = model_7.predict(X_train_eda_2)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("EDA Model 2 Training RMSE: ", rmse)

In [ ]:
# Ridge regression tended to lower RMSE previously, so we can also try ridge for the above 2 models.
#model_8 = RidgeCV(cv=5, alphas=np.arange(0, 10.01, 0.1))
#model_8.fit(X_train_eda_1, y_train)
#y_pred = model_8.predict(X_train_eda_1)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("EDA Ridge Model 1 Training RMSE: ", rmse)

In [ ]:
#model_9 = RidgeCV(cv=5, alphas=np.arange(0, 10.01, 0.1))
#model_9.fit(X_train_eda_2, y_train)
#y_pred = model_9.predict(X_train_eda_2)
#rmse = mean_squared_error(y_train, y_pred) ** 0.5
#print("EDA Ridge Model 2 Training RMSE: ", rmse)

In [ ]:
#from sklearn.model_selection import cross_val_score
#import sklearn
# Now we can do cross validation on our models and find the best performing one
#models_q6 = [
#(base_model, X_train),
#(model_1, X_train_lasso),
#(model_2, X_train),
#(model_3, X_train_no_ts),
#(model_4, X_train_lasso_no_ts),
#(model_5, X_train_no_ts),
#(model_6, X_train_eda_1),
#(model_7, X_train_eda_2),
#(model_8, X_train_eda_1),
#(model_9, X_train_eda_2)
#]
#scores = []
#count = 1
#for pair in models_q6:
#    model = pair[0]
#    features = pair[1]
#    cur_model_scores = cross_val_score(model, features, y_train, scoring='neg_root_mean_squared_error', cv=5)
#    scores.append((cur_model_scores.mean() * -1))
#    print("Finished scoring model {} out of {}".format(count, len(models)))
#    count += 1
#model_score_pairs = []
#for i in range(len(scores)):
#    model_score_pairs.append((i, scores[i]))
#model_score_pairs.sort(key = lambda x: x[1])
#print("Model Rankings (Best to Worst)")
#print("==================================")
#for pair in model_score_pairs:
#    num = pair[0]
#    rmse = pair[1]
#    if num == 0:
#        name = 'base_model'
#    else:
#        name = 'model_{}'.format(num)
#    print('{} RMSE: {}'.format(name, rmse))


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6c

If applicable, write an equation or rule for the prediction function $f(x)$; if this is infeasible, make sure to visualize your model parameters in some way. Interpret your improved model's optimal parameters (*hint*: refer to 1aiii), and compare these parameters to those of the baseline model. Comment on whether the parameters follow physical intuition given the nature of the prediction task.

For example, if you chose to use a decision tree, you may interpret the generated rules.
 
<!--
BEGIN QUESTION
name: q6c
points: 0
manual: True
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6d

Discuss your improved model's performance on both short-term and long-term time scales using a metric of your choice (*hint:* we're using an autoregressive model). In other words, given $x_t$, we wish to predict $\hat{x}_{t+k}$, and plot the error of these predictions for two $k$ values of your choice. You may use any reasonable interpretation of short-term and long-term predictions; an initial suggestion is to use 2-day predictions and 2-week predictions.

Compare the performance of this model on both timescales with the baseline model.

<!--
BEGIN QUESTION
name: q6d
points: 0
manual: True
-->

In [ ]:
#wanted_features = [
#'POPESTIMATE2020',
#'Lat',
#'Long_',
#'9/12/21',
#'NEVER',
#'RARELY',
#'SOMETIMES',
#'FREQUENTLY',
#'ALWAYS',
#'People_Fully_Vaccinated',
#'People_Partially_Vaccinated'
#]
#with_preds = merged.copy()
#with_preds[wanted_features]
#y_pred = model_3.predict(with_preds[wanted_features])
#with_preds['predictions'] = y_pred
#modded_fips = []
#for fip in with_preds['county_fips']:
#    fip = str(fip)
#    if len(fip) < 5:
#        fip = '0' + fip
#    modded_fips.append(fip)
#with_preds['county_fips'] = modded_fips

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6e

Plot and describe the error for both the baseline and improved models as a function of time. In other words, given $x_t$, we wish to predict $\hat{x}_{t+k}$, and plot the error of these predictions for all $k$.

Consider how and why the performance of the model degrades as a function of time using the rate of growth in the error.

<!--
BEGIN QUESTION
name: q6e
points: 0
manual: True
-->

In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 6f

Consider a modification to the model $f(x) = x_{t+1}$ where instead $f(x) = [x_{t+1}, x_{t+2}, ..., x_{t+m}]$ for some $m > 1$. In other words, using the features $x$ that contain past and present quantities, our model *explicitly* predicts values for $m$ days in the future rather than simply the next day (i.e. $m = 1$). 

Train the baseline and improved model using $m = 5$ and $m = 10$. Evaluate and visualize the predictive accuracy of both models.

<!--
BEGIN QUESTION
name: q6f
points: 0
manual: True
-->

In [ ]:
...

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()